# Scope
---
In this notebook, I'll describe
- name_scope
- variable_scope

## [tf.name_scope](https://www.tensorflow.org/api_docs/python/tf/name_scope)
The `tf.name_scope` defines the group of some variables in a graph.  
It add a prefix to the operation or variable made by using `tf.Variable`.  
__Note that the `tf.name_scope` is ignore by `tf.get_variable`.__  
Some google developer recommends that we should use variable_scope and not use name_scope because of the reason above.

## [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope)
> A context manager for defining ops that creates variables (layers).
This context manager validates that the (optional) values are from the same graph, ensures that graph is the default graph, and pushes a name scope and a variable scope.
If name_or_scope is not None, it is used as is. If name_or_scope is None, then default_name is used. In that case, if the same name has been previously used in the same scope, it will be made unique by appending _N to it.

## tf.name_scope vs. tf.variable_scope
The concepts of both scope are the same. They define the prefix of the operation or variable. Both scopes have the same effect on all operations as well as variables created using tf.Variable, i.e., the scope will be added as a prefix to the operation or variable name.

In [1]:
import tensorflow as tf
print("tensorflow version: ", tf.__version__)

tensorflow version:  1.12.0


## Examples
### - `tf.name_scope` / `tf.variable_scope`
`tf.get_variable` ignores the scope defined by `tf.name_scope()`

In [2]:
# clear graph
tf.reset_default_graph()

with tf.name_scope("name_scope"):
    # tf.Variable()
    v1 = tf.Variable([1,2,3], name="var1", dtype=tf.float32)
    # tf.get_variable()
    v2 = tf.get_variable("var2", shape=[3,], dtype=tf.float32)
    # define ops
    add = tf.add(v1, v2)

print(v1.name)
print(v2.name)
print(add.name)

name_scope/var1:0
var2:0
name_scope/Add:0


`tf.get_variable` doesn't ignore the scope defined by `tf.variable_scope`

In [3]:
# clear graph
tf.reset_default_graph()

with tf.variable_scope("variable_scope"):
    # tf.Variable()
    v1 = tf.Variable([1,2,3], name="var1", dtype=tf.float32)
    # tf.get_variable()
    v2 = tf.get_variable("var2", shape=[3,], dtype=tf.float32)
    # define ops
    add = tf.add(v1, v2)

print(v1.name)
print(v2.name)
print(add.name)

variable_scope/var1:0
variable_scope/var2:0
variable_scope/Add:0


Duplication of the variable name.

In [4]:
# clear graph
tf.reset_default_graph()

with tf.variable_scope("variable_scope"):
    v1 = tf.Variable([1,2,3], name="var1", dtype=tf.float32)
    v2 = tf.get_variable("var2", shape=[3,], dtype=tf.float32)
    add = tf.add(v1, v2)

print(v1.name)
print(v2.name)
print(add.name)
print("\n")

with tf.variable_scope("variable_scope"):
    # The scope name will be variable_scope_1 because variable_scope/v1 is already exists.
    v1 = tf.Variable([1,2,3], name="var1", dtype=tf.float32)
    # The var3 will be added to "variable_scope".
    v3 = tf.get_variable("var3", shape=[3,], dtype=tf.float32)
    # The scope name will be variable_scope_1 because variable_scope/add is already exists.
    add = tf.add(v1, v3)
    
print(v1.name)
print(v3.name)
print(add.name)

variable_scope/var1:0
variable_scope/var2:0
variable_scope/Add:0


variable_scope_1/var1:0
variable_scope/var3:0
variable_scope_1/Add:0


### [Sharing variables](https://www.tensorflow.org/guide/variables)
Example: Sharing variable `w` in the calculation bellow.
- output1 = input_A $\cdot$ w
- output2 = input_B $\cdot$ w
- final_output = output1 + output2

Failure Case:

In [7]:
# clear graph
tf.reset_default_graph()

input_A = tf.placeholder(tf.float32, shape=[3,1], name="input_A")
input_B = tf.placeholder(tf.float32, shape=[3,1], name="input_B")

# define ops
def matmul(input_vector):
    with tf.variable_scope("matmul"):
        w = tf.get_variable("weight", shape=(1,3), initializer=tf.initializers.ones)
        return tf.matmul(input_vector, w)

output1 = matmul(input_A)
output2 = matmul(input_B) # This will be error because matmul/weight already exists.
final_output = tf.add(output1, output2)

ValueError: Variable matmul/weight already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\Users\yoshiyuki\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\ops.py", line 1770, in __init__
    self._traceback = tf_stack.extract_stack()
  File "C:\Users\yoshiyuki\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\ops.py", line 3274, in create_op
    op_def=op_def)
  File "C:\Users\yoshiyuki\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\util\deprecation.py", line 488, in new_func
    return func(*args, **kwargs)


Success case:

In [10]:
# clear graph
tf.reset_default_graph()

input_A = tf.placeholder(tf.float32, shape=[3,1], name="input_A")
input_B = tf.placeholder(tf.float32, shape=[3,1], name="input_B")

# define ops
def matmul(input_vector, reuse = False):
    with tf.variable_scope("matmul", reuse=reuse):
        w = tf.get_variable("weight", shape=(1,3), initializer=tf.initializers.ones)
        print(w.name)
        return tf.matmul(input_vector, w)

output1 = matmul(input_A)
output2 = matmul(input_B, reuse = True) # This will be error because matmul/weight already exists.
final_output = tf.add(output1, output2)

matmul/weight:0
matmul/weight:0
